# Example: Analyzing Citibike Station Activity using Pandas

We are going to use the database of snapshots of Citibike stations statuses. 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

In [ ]:
!sudo pip3 install -U sklearn geopandas

## Fetching Data

We will first fetch the data from the database. The original SQL query on the raw data is the following, and is computing the average capacity of a bike station in hourly intervals.

```sql
    SELECT S.station_id AS id, 
           AVG(num_bikes_available) AS available_bikes,
           AVG(num_docks_available) AS available_docks,
           AVG(num_bikes_disabled) AS disabled_bikes,
           AVG(num_docks_disabled) AS disabled_docks,
           T.capacity, T.lat, T.lon, T.name,
           DATE_FORMAT(last_reported, '%%Y-%%m-%%d %%H:00:00') AS communication_time 
    FROM citibike_new.Status S JOIN citibike_new.Stations T ON T.station_id = S.station_id
    WHERE  is_renting=1 AND is_installed=1 AND T.capacity>0 AND last_reported>='2017-10-01' AND last_reported<='2017-11-15'
    GROUP BY S.station_id, communication_time
```

For speed, we store the results of this query in a table.

In [ ]:
%%time
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/{database}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015',
    database = 'citibike_fall2017',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

query = '''
SELECT *
FROM status_fall2017
'''
df = pd.read_sql(query, con=engine)
df.communication_time = pd.to_datetime(df.communication_time, format='%Y-%m-%d %H:%M:%S')


In [ ]:
len(df)

In [ ]:
df.head(5)

### Exploratory Analysis

As a first step, let's see how the status of the bike stations evolves over time. We compute the average "**fullness**" of **all** the bike stations over time.

In [ ]:
df['percent_full'] = df.available_bikes / df.capacity

Now let's plot the activity over time. We can see that the percentage of bikes in the stations fluctuates over time, with around 10% difference from the daily-lows to the daily-highs. 

In [ ]:
timeseries = df.pivot_table(
    index='communication_time', 
    values='percent_full',
    aggfunc='mean'
)

timeseries.plot( figsize=(15, 7), ylim=(0.25,0.55), xlim=('2017-10-01', '2017-10-08'))

Let's do also the seasonal decomposition to see the result.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(timeseries, model='multiplicative', freq=24)
seasonal = decomposition.plot()  
seasonal.set_size_inches(20, 15)

### Examining Time Series per Station

We now create a pivot table, to examine the time series for individual stations.

Notice that we use the `interpolate` method, where we fill the cells using interpolation using the non-missing values before and after.

In [ ]:
station_timeseries = df.pivot_table(
    index='communication_time', 
    columns='id', 
    values='percent_full', 
    aggfunc='mean'
).interpolate(method='time') 

station_timeseries.head(5)

Let's plot the time series for *all* bike stations, for a few days.

In [ ]:
station_timeseries.plot(
    alpha=0.1, 
    color='b', 
    legend=False, 
    figsize=(20,5), 
    xlim=('2017-10-01','2017-10-07'),
    ylim=(0,1)
)

Let's limit our plot to just two stations:
* Station 3260 at "Mercer St & Bleecker St"
* Station 161 at "LaGuardia Pl & W 3 St"

which are nearby and tend to exhibit similar behavior. Remember that the list of stations is [available as a JSON](https://feeds.citibikenyc.com/stations/stations.json) 

In [ ]:
# df [ df.name.str.contains("Mercer") & df.name.str.contains("Bleecker") ]
# df [ df.name.str.contains("LaGuardia") ]

In [ ]:
station_timeseries[ [161, 3260] ].plot(
    alpha=0.5,  
    legend=False, 
    figsize=(20,5), 
    xlim=('2017-10-11','2017-10-14'),
    ylim=(0,1)
)

## Finding Bike Stations with Similar Behavior

For our next analysis, we are going to try to find bike stations that have similar behaviors over time. A very simple technique that we can use to find similar time series is to treat the time series as vectors, and compute their correlation. Pandas provides the `corr` function that can be used to calculate the correlation of columns. (If we want to compute the correlation of rows, we can just take the transpose of the dataframe using the `transpose()` function, and compute the correlations there.)

In [ ]:
similarities = station_timeseries.corr(method='pearson')
similarities.head(5)

Let's see the similarities of the two stations that we examined above.

In [ ]:
stations = [161, 3260]

similarities[stations].loc[stations]

In [ ]:
# 393: E 5 St & Avenue C
# 2003: 1 Ave & E 18 St
stations = [393, 2003] 
    
similarities[stations].loc[stations]

In [ ]:
station_timeseries[ [393, 2003]  ].plot(
    alpha=0.5,  
    legend=False, 
    figsize=(20,5), 
    xlim=('2017-10-01','2017-10-14'),
    ylim=(0,1)
)

For bookkeeping purposes, we are going to drop stations that generate NaN values, as we cannot use such entries for our analysis.

In [ ]:
# Number of stations with non-NaN similarity per station
check = similarities.count()
# Find the number of stations with less than the max number of similarities
todrop = check[check < check.max()].index.values
similarities.drop(todrop, axis='index', inplace=True)
similarities.drop(todrop, axis='columns', inplace=True)

### Clustering Based on Distances

Without explaining too much about clustering, we are going to use a clustering technique and cluster together bike stations that are "nearby" according to our similarity analysis. For this, we need to first convert our similarities to distance.

We are now going to convert our **similarities** into **distance** metrics. Our distance values will be always positive, and bounded between 0 and 1.

* If two stations have correlation 1, they behave identically, and therefore have distance 0, 
* If two stations have correlation -1, they have exactly the oppositite behaviors, and therefore we want to have distance 1 (the max) 

In [ ]:
# Similarity goes from -1 to 1, so 1-similarity goes from 0 to 2.
# So, we multiply with 0.5 to get it between 0 and 1, and then take the square
distances = ((.5*(1-similarities))**2)
distances.head(5)

The clustering code is very simple: The code below will create two groups of stations.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=2)
cluster.fit(distances.values)

We will now take the results of the clustering and associate each of the data points into a cluster.

In [ ]:
labels = pd.DataFrame(list(zip(distances.index.values.tolist(), cluster.labels_)), columns = ["id", "cluster"])
labels.head(10)

Let's see how many stations in each cluster

In [ ]:
labels.pivot_table(
    index = 'cluster',
    aggfunc = 'count'
)

### Visualizing the Time Series Clusters

We will start by assining a color to each cluster, so that we can plot each station-timeline with the cluster color. (We put a long list of colors, so that we can play with the number of clusters in the earlier code, and still get nicely colored results.)

In [ ]:
colors = list(['red','blue', 'green', 'magenta', 'yellow', 'black', 'white', 'cyan'])
labels['color'] = labels['cluster'].apply(lambda cluster_id : colors[cluster_id]) 
labels.head(10)

In [ ]:
stations_plot = station_timeseries.plot(
    alpha=0.1, 
    legend=False, 
    figsize=(20,5), 
    linewidth=1,
    color=labels["color"],
    xlim=('2017-10-01','2017-10-07'),
    ylim=(0,1)
)

The plot still looks messy. Let's try to plot instead a single line for each cluster. To represent the cluster, we are going to use the _median_ fullness value across all stations that belong to a cluster, for each timestamp. For that, we can again use a pivot table: we define the `communication_time` as one dimension of the table, and `cluster` as the other dimension, and we use the `median` function. 

For that, we first _join_ our original dataframe with the results of the clustering, using the `merge` command, and add an extra column that includes the clusterid for each station. Then we compute the pivot table.

In [ ]:
median_cluster = df.merge(
    labels, 
    how='inner', 
    on='id'
).pivot_table(
    index='communication_time', 
    columns='cluster', 
    values='percent_full', 
    aggfunc='median'
)

median_cluster.head(15)

Now, we can plot the medians for the two clusters.

In [ ]:
median_cluster.plot(
    figsize=(20,5), 
    linewidth = 2, 
    alpha = 0.75,
    color=colors,
    ylim = (0,1),
    xlim=('2017-10-01','2017-10-10'),
    grid = True
)

And just for fun and for visual decoration, let's put the two plots together. We are going to fade a lot the individual station time series (by putting the `alpha=0.005`) and we are going to make more prominent the median lines by increasing their linewidths. We will limit our plot to one week's worth of data:

In [ ]:
stations_plot = station_timeseries.plot(
    alpha=0.005, 
    legend=False, 
    figsize=(20,5), 
    color=labels["color"]
)

median_cluster.plot(
    figsize=(20,5), 
    linewidth = 3, 
    alpha = 0.5,
    color=colors, 
    xlim=('2017-10-01','2017-10-14'),
    ylim=(0,1),
    ax = stations_plot
)

### Mapping

Now, we will plot the stations in a map, to see where the clusters are located.

Just to make things more interesting, we will also get information about the current status of the stations.

In [ ]:
# Get information about current (real-time) status of the stations 
import requests

url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'
results = requests.get(url_status).json() 
status = results["data"]["stations"]
df_status = pd.DataFrame(status)
df_status = df_status[ ['station_id', 'num_bikes_available'] ] #keep only station id and bikes available
df_status.columns = ['id', 'bikes_available'] # rename the columns
df_status.id = pd.to_numeric(df_status.id) # convert id to numeric
df_status.head(5)

In [ ]:
df_status.dtypes

In [ ]:
# Keep only the necessary columns / rows
map_df = df[ ['id', 'lon', 'lat', 'capacity'] ].drop_duplicates()
# Join the list of stations with the cluster
map_df = map_df.merge(
    labels, 
    how='inner', 
    on='id'
)
# Join the list of stations with the current status
map_df = map_df.merge(
    df_status, 
    how='inner', 
    on='id'
)
map_df = map_df.set_index('id')
# Reorder the columns
map_df = map_df[ ['cluster', 'capacity', 'color', 'lon', 'lat', 'bikes_available'] ] 

In [ ]:
map_df.head(5)

In [ ]:
# Plot the stations using a scatterplot with their lon/lat values
# Use a different color for each cluster
# Change the size of each dot according to the capacity
map_df.plot.scatter(
    x = 'lon',
    y = 'lat',
    edgecolors='Black',
    c = map_df.color.values,
    s = 3*map_df.bikes_available.values,
    figsize=(10,10), alpha=0.6
)


In [ ]:
# We draw a background with the map of the NYC neighborhoods
import geopandas as gpd

# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
shapefile = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=Shapefile'
df_nyc = gpd.GeoDataFrame.from_file(shapefile)
# Plot the NYC neighborhoods map
base = df_nyc.plot(
    linewidth=0.5,
    color='White',
    edgecolor='Black',
    figsize=(10, 10),
    alpha=0.5)
base.set_xlim((-74.10, -73.92))
base.set_ylim((40.65, 40.82))

map_df.plot.scatter(
    x='lon',
    y='lat',
    edgecolors='Black',
    c=map_df.color.values,
    s=3 * map_df.bikes_available.values,
    figsize=(10, 10),
    alpha=0.6,
    ax=base)

### Mapping with Folium

In [ ]:
import folium

map_center = [map_df.lat.median(), map_df.lon.median()]
fmap = folium.Map(location=map_center, zoom_start=14,  tiles='cartodbpositron')
fmap

In [ ]:
map_center = [map_df.lat.median(), map_df.lon.median()]
fmap = folium.Map(location=map_center, zoom_start=13,  tiles='cartodbpositron')

for name, row in map_df.iterrows():
    
    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    # opacity = row["bikes_available"]/row["capacity"]
    # Make the color green for the working stations, red otherwise
    color = row["color"]
    # The size of the marker is proportional to the number of docks
    size1 = row["capacity"]/7+1
    size2 = row["bikes_available"]/7+1

    
    # We create a marker on the map with size proportional to its capacity
    folium.CircleMarker(location=[row["lat"], row["lon"]], 
                        radius = size1,
                        color='black', weight=1, 
                        fill=True,
                        fill_opacity = 0.25,
                        fill_color = color,
                       ).add_to(fmap)
    
    # We create a marker with size proportional to the number of bikes 
    # This will be the "internal" circle, indicating how full the station is
    # We make the opacity higher
    folium.CircleMarker(location=[row["lat"], row["lon"]], 
                        radius = size2,
                        color='black', weight=0.5, 
                        fill=True,
                        fill_opacity = 0.75,
                        fill_color=color,
                       ).add_to(fmap)
    


In [ ]:
fmap

In [ ]:
# Save the map as HTML (citibike.html). Also add the NYC neighborhoods 

# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
url = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON'
nyc_neighborhoods = gpd.GeoDataFrame.from_file(url)
style_function = lambda x: {'fillOpacity': 0.0, 'weight': 0.5}
folium.GeoJson(
    nyc_neighborhoods,
    name='geojson', 
    style_function=style_function
).add_to(fmap)

import os
fn='citibike.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
fmap.save(fn)